# bert_score

In [4]:
from bert_score import score
import json

# Load your JSON
with open('test_questions.json', 'r') as f:
    data = json.load(f)

# Extract AI and GT answers
ai_answers = [item['AI'] for item in data]
gt_answers = [item['answer'] for item in data]

# Calculate BERTScore
P, R, F1 = score(ai_answers, gt_answers, lang="en", verbose=True)

# Print average
print(f"Average Precision: {P.mean().item():.4f}")
print(f"Average Recall: {R.mean().item():.4f}")
print(f"Average F1: {F1.mean().item():.4f}")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 24.97 seconds, 0.60 sentences/sec
Average Precision: 0.9031
Average Recall: 0.8713
Average F1: 0.8864


# Rouge

In [6]:
from rouge import Rouge
import json

# Load your JSON
with open('test_questions.json', 'r') as f:
    data = json.load(f)

ai_answers = [item['AI'] for item in data]
gt_answers = [item['answer'] for item in data]

# Initialize Rouge
rouge = Rouge()

# Calculate ROUGE-L
scores = [rouge.get_scores(ai, gt)[0]['rouge-l']['f'] for ai, gt in zip(ai_answers, gt_answers)]

# Print average ROUGE-L score
average_rouge_l = sum(scores) / len(scores)
print(f"Average ROUGE-L F1 Score: {average_rouge_l:.4f}")


Average ROUGE-L F1 Score: 0.2653


# Bleu and Rouge

In [10]:
# Install needed libraries first (uncomment if you haven't installed them)
# pip install bert_score
# pip install rouge
# pip install nltk

from bert_score import score as bert_score
from rouge import Rouge
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import json
import statistics
import time

# Download punkt if not already
nltk.download('punkt')

# Load your JSON data
with open('test_questions.json', 'r') as f:
    data = json.load(f)

# Extract AI and GT answers
ai_answers = [item['AI'] for item in data]
gt_answers = [item['answer'] for item in data]

# Start timer for latency
start_time = time.time()


# -------- ROUGE-L --------
print("\nCalculating ROUGE-L...")
rouge = Rouge()
rouge_l_f1_scores = []
for ai, gt in zip(ai_answers, gt_answers):
    try:
        score = rouge.get_scores(ai, gt)[0]['rouge-l']['f']
        rouge_l_f1_scores.append(score)
    except ValueError:
        # If AI or GT is empty, skip
        rouge_l_f1_scores.append(0.0)
average_rouge_l_f1 = statistics.mean(rouge_l_f1_scores)

# -------- BLEU Score --------
print("\nCalculating BLEU Score...")
# Prepare BLEU inputs
references = [[gt.split()] for gt in gt_answers]  # Each GT wrapped in a list
candidates = [ai.split() for ai in ai_answers]
# Smoothing for better BLEU with short sentences
smooth_fn = SmoothingFunction().method4
bleu_score_value = corpus_bleu(references, candidates, smoothing_function=smooth_fn)

# End timer for latency
end_time = time.time()
total_time = end_time - start_time
average_latency = total_time / len(data)

# -------- Results --------
print("\n--- Evaluation Results ---")

print(f"ROUGE-L F1 Score: {average_rouge_l_f1:.4f}")
print(f"BLEU Score: {bleu_score_value:.4f}")
print(f"Average Latency per QA: {average_latency:.2f} seconds")

# Optional: Save results to a file
results = {
   
    "ROUGE_L_F1": average_rouge_l_f1,
    "BLEU_Score": bleu_score_value,
    "Average_Latency_per_QA": average_latency,
    "Total_QA_Pairs": len(data)
}

with open('evaluation_results.json', 'w') as f:
    json.dump(results, f, indent=4)

print("\nResults saved to evaluation_results.json ✅")



Calculating ROUGE-L...

Calculating BLEU Score...

--- Evaluation Results ---
ROUGE-L F1 Score: 0.2653
BLEU Score: 0.0438
Average Latency per QA: 0.00 seconds

Results saved to evaluation_results.json ✅


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ujasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# cosine similarity and rouge

In [16]:

# Load the data from x.json file
with open('test_questions.json', 'r') as f:
    data = json.load(f)

# Assuming that the data is a list and we need to access the first item in it
entry = data[0]  # Accessing the first dictionary in the list

# Extracting the real and AI answers from the first entry
real_answer = entry["answer"]
ai_answer = entry["AI"]

# Vectorizing the text using TF-IDF
vectorizer = TfidfVectorizer().fit_transform([real_answer, ai_answer])
cosine_sim = cosine_similarity(vectorizer[0:1], vectorizer[1:2])

# ROUGE score (recall-based metric)
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(real_answer, ai_answer)

# Word Overlap / Exact Match (counting matching words)
exact_match = len(set(real_answer.lower().split()).intersection(set(ai_answer.lower().split())))

# Output the results
print(f"Cosine Similarity: {cosine_sim[0][0]:.4f}")
print(f"ROUGE Scores: {rouge_scores}")


Cosine Similarity: 0.3345
ROUGE Scores: {'rouge1': Score(precision=0.5, recall=0.2682926829268293, fmeasure=0.34920634920634924), 'rouge2': Score(precision=0.2857142857142857, recall=0.15, fmeasure=0.1967213114754098), 'rougeL': Score(precision=0.4090909090909091, recall=0.21951219512195122, fmeasure=0.28571428571428575)}
